In [1]:
#Geophysique CEA
# LIBRAIRIES Python
# Pyton 2.7 Numpy 1.14.5 Matplotlib 2.2.3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import seaborn as sns

# For model construction and data processing
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

In [2]:
#MM
import os
os.getcwd()

'c:\\Users\\hasso\\PythonWithAnaconda\\GEO_CEA\\Article'

# Loading and processing data

In [4]:
## Loading training HR
data_train = np.loadtxt('./DATA/dataHR20K.DAT')
X = np.copy(data_train[:,0:3])

# Data and processing
scaler = MinMaxScaler().fit(X)

# Scaling
X_Train_M13 = scaler.transform(X) 
Y_Train_M13 = np.copy(data_train[:,3])

# Testing data
## Test M13
data_test_M13 = np.loadtxt('./DATA/dataVALID_interHR.DAT')
X_1 = np.copy(data_test_M13[:,0:3])
X_test_M13 = scaler.transform(X_1)   
Y_test_M13 = np.copy(data_test_M13[:,3])

## Test L
data_test_M13_L = np.loadtxt('./DATA/dataVALIDL_interLR.DAT')
X_L = np.copy(data_test_M13_L[:,0:3])
X_test_M13_L = scaler.transform(X_L)
Y_test_M13_L = np.copy(data_test_M13_L[:,3])

# Model setup and Cross-validation

In [5]:
# Set up initial model
tree_mod = tree.DecisionTreeRegressor(min_samples_leaf=1)
knn = KNeighborsRegressor(n_neighbors=2)
Bag = BaggingRegressor(base_estimator=tree_mod)
RF = RandomForestRegressor(min_samples_leaf=1)

In [6]:
# Cross-validation 
# ================
from sklearn.model_selection import GridSearchCV

#create a dictionary of all values we want to test for n_neighbors
# ================================================================

# param_grid_rf = {"n_estimators" : np.arange(10,151,10)}
param_grid_knn = {"n_neighbors" : np.arange(1,20)}

#use gridsearch to test all values for n_neighbors
# ================================================

# bag_cv = GridSearchCV(Bag, param_grid_rf, cv=20)
# rf_cv = GridSearchCV(RF, param_grid_rf, cv=20)
knn_cv = GridSearchCV(knn, param_grid_knn, cv=20)

#fit model to data
# ================

# bag_cv.fit(X_Train_M13, Y_Train_M13)
# rf_cv.fit(X_Train_M13, Y_Train_M13)
knn_cv.fit(X_Train_M13, Y_Train_M13)

GridSearchCV(cv=20, estimator=KNeighborsRegressor(n_neighbors=2),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [8]:
# Best parameters
# ===============
# bag_cv.best_params_
# rf_cv.best_params_

knn_cv.best_params_

{'n_neighbors': 2}

# Training and prediction

In [9]:
# Training and prediction
# =======================

## Optimal model

# Bag = BaggingRegressor(n_estimators = bag_cv.best_params_["n_estimators"], base_estimator=tree_mod)
# RF = RandomForestRegressor(n_estimators = rf_cv.best_params_["n_estimators"], min_samples_leaf=1)
knn = KNeighborsRegressor(n_neighbors = knn_cv.best_params_['n_neighbors'])

## Training and prediction

### Tree 
# treefit = tree_mod.fit(X_Train_M13,Y_Train_M13)
# pred_train_tr = treefit.predict(X_Train_M13)
# pred_test_tr = treefit.predict(X_test_M13)
# pred_test_tr_L = treefit.predict(X_test_M13_L)

### Bagging
# Bagfit = Bag.fit(X_Train_M13, Y_Train_M13)
# pred_train_bag = Bagfit.predict(X_Train_M13)
# pred_test_bag = Bagfit.predict(X_test_M13)
# pred_test_bag_L = Bagfit.predict(X_test_M13_L)

### Random Forest
# RFfit = RF.fit(X_Train_M13, X_Train_M13)
# pred_train_rf = RFfit.predict(X_Train_M13)
# pred_test_rf = RFfit.predict(X_test_M13)
# pred_test_rf_L = RFfit.predict(X_test_M13_L)

### KNN

KNNfit = knn.fit(X_Train_M13, Y_Train_M13)
pred_train_knn = KNNfit.predict(X_Train_M13)
pred_test_knn = KNNfit.predict(X_test_M13)
pred_test_knn_L = KNNfit.predict(X_test_M13_L)

#print('Testing Error of Tree on M13 data:' + str(metrics.mean_absolute_error(pred_test_tr, Y_test_M13, squared=True)))
#print('Testing Error of Tree on M13_L data:' + str(metrics.mean_absolute_error(pred_test_tr_L, Y_test_M13_L, squared=True)))
#print('Testing Error of Bag M13 data:' + str(metrics.mean_absolute_error(pred_test_bag, Y_test_M13, squared=True)))
#print('Testing Error of Bag M13_L data:' + str(metrics.mean_absolute_error(pred_test_bag_L, Y_test_M13_L, squared=True)))
#print('Testing Error of RF on M13 data:' + str(metrics.mean_absolute_error(pred_test_rf, Y_test_M13, squared=True)))
#print('Testing Error of RF on M13_L data:' + str(metrics.mean_absolute_error(pred_test_rf_L, Y_test_M13_L, squared=True)))
print('Testing Error of KNN on M13 data:' + str(metrics.mean_absolute_error(pred_test_knn, Y_test_M13)))
print('Testing Error of KNN on M13_L data:' + str(metrics.mean_absolute_error(pred_test_knn_L, Y_test_M13_L)))

Testing Error of KNN on M13 data:0.010126926977687639
Testing Error of KNN on M13_L data:0.36371843036978757
